In [220]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
# from torchsummary import summary
from torchinfo import summary

In [221]:
class MultiScaleStockDataset(Dataset):
    """多模尺度数据集"""
    def __init__(self, scale_factors, seq_length=24, pred_length=4, data_path='./'):
        self.standardScaler = StandardScaler()
        self.scale_factors = scale_factors
        self.seq_length = seq_length
        self.pred_length = pred_length
        self.data_path = data_path
        self.data = {
            scale: self._generate_synthetic_data(scale) for scale in self.scale_factors
        }
        # 验证数据维度
        for scale, data in self.data.items():
            assert len(data) > seq_length + pred_length, \
                f"{scale}数据长度不足，需要至少{seq_length + pred_length}个样本"
            assert data.shape[1] == 4, \
                f"{scale}数据特征维度应为4，实际为{data.shape[1]}"
    
    def _generate_synthetic_data(self, scale):
        """获取合成数据"""
        data = pd.read_csv(self.data_path + f'train-{scale}.csv', 
                           usecols=['date','open', 'high', 'low', 'close'],
                           parse_dates=['date'], index_col='date')
        return self.standardScaler.fit_transform(data)
    
    def _align_scales(self, idx):
        """三线性插值对齐时间轴"""
        aligned = {}
        base_idx = idx // 8  # 对齐到日线级别
        for scale in self.scale_factors:
            ratio = {'30min':48, '1hour':24, '4hour':6, '1day':1}[scale]
            # start = int(base_idx * ratio)
            scale_idx = int(base_idx * ratio)
            seq_items = int(self.seq_length * ratio)

            aligned_seq = self.data[scale][scale_idx:scale_idx + seq_items]
            # 添加空数据检查
            if len(aligned_seq) == 0:
                aligned_seq = np.zeros((self.seq_length, 4))  # 用零填充
            # Linear interpolation if needed
            if len(aligned_seq) < self.seq_length:
                x_orig = np.arange(len(aligned_seq))
                x_new = np.linspace(0, len(aligned_seq)-1, self.seq_length)
                aligned_seq = np.array([np.interp(x_new, x_orig, col) 
                                    for col in aligned_seq.T]).T
                
            aligned[scale] = aligned_seq[:self.seq_length]
        return aligned
    
    def __getitem__(self, idx):
        inputs = self._align_scales(idx)
         # 确保填充后的维度一致性
        for k,v in inputs.items():
            assert v.shape[1] == 4, f"特征维度错误: {k}的维度为{v.shape[1]}"

        # target = self.data['30min'][idx:idx+self.pred_length, 0]  # 预测未来价格
         # 统一所有尺度特征维度为4
        aligned = {k: np.pad(v, ((0,0),(0,1))) if v.shape[1]==3 else v 
              for k,v in inputs.items()}
        target = self.data['30min'][idx+self.pred_length-1, 0]  # 从[4]变为标量
        # 双重验证
        assert isinstance(target, np.float64), "目标值类型错误"
        target_tensor = torch.tensor(target, dtype=torch.float32)
        return {k:torch.FloatTensor(v) for k,v in aligned.items()}, target_tensor
    def __len__(self):
        return len(self.data[self.scale_factors[0]]) - self.pred_length - self.seq_length

class ScaleAwareExpert(nn.Module):
    """专家"""
    def __init__(self, input_dim, scale_type):
        super().__init__()
        self.scale_type = scale_type

        if '30min' in scale_type or '1h' in scale_type:
            """高频专家: CNN + BiLSTM """
            self.conv = nn.Conv1d(input_dim,
                                  out_channels=16,
                                  kernel_size=3, 
                                  padding='same', 
                                  )
            self.lstm = nn.LSTM(16,32, num_layers=2, batch_first=True, bidirectional=True)
            self.proj = nn.Linear(64,32)
        else:
            """低频专家: LSTM """
            self.proj_in = nn.Linear(input_dim, 4)  # 3维→4维
            self.encoder_layer = nn.TransformerEncoderLayer(d_model=4, nhead=2, dim_feedforward=64, batch_first=True)
            self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
            self.proj = nn.Linear(4, 32)
        self.predictor = nn.Linear(32, 1)

    def forward(self, x):
        if '30min' in self.scale_type or '1h' in self.scale_type:
            x = self.conv(x.permute(0,2,1)).permute(0,2,1)
            x, _ = self.lstm(x)
            x = x[:, -1, :]  # 取最后一个时间步 [batch, 4]
            x = self.proj(x)
        else:
            x = self.proj_in(x)
            x = self.transformer(x)
            x = self.proj(x[:,-1,:]) # 只取最后一个时间步的输出
        return self.predictor(x)
class MoE(nn.Module):
    """混合专家网络"""
    def __init__(self, input_dim=4, experts=['30min','1hour','4hour','1day'], hidden_dim=64, output_dim=1):
        super().__init__()
        self.experts = nn.ModuleDict({
            scale: ScaleAwareExpert(input_dim, scale) for scale in experts
        })
        self.gate = nn.Sequential(
            nn.Linear(input_dim * len(experts), hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, len(experts)),
            nn.Softmax(dim=-1)
        )

    def forward(self, inputs):
        expert_outputs = {}
        for scale, expert in self.experts.items():
            # expert_outputs[scale] = expert(inputs[scale])
            expert_outputs[scale] = expert(inputs[scale]).squeeze(-1)  # [32,1] -> [32]
        # 动态门控
        gate_input = torch.cat([v for v in inputs.values()], dim=-1)
        weights = self.gate(gate_input.mean(dim=1))  # (batch, num_experts)
        
        # 加权融合
        combined = sum(weights[:, i] * expert_outputs[scale] 
                      for i, scale in enumerate(self.experts.keys()))
        return combined.unsqueeze(-1), expert_outputs  

In [222]:
# ==================== 评估指标 ==================== 
def sharpe_ratio(returns, risk_free=0.0):
    excess_returns = returns - risk_free
    return excess_returns.mean() / excess_returns.std()

def max_drawdown(returns):
    cumulative = returns.cumsum()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (peak - cumulative).max()
    return drawdown

In [223]:
def test_conv_layer():
    # 模拟输入 (batch=2, seq=24, features=4)
    test_input = torch.randn(2, 24, 4)
    expert = ScaleAwareExpert(input_dim=4, scale_type='30min')
    
    # 添加维度跟踪
    print("=== 维度跟踪 ===")
    x = expert.conv(test_input.permute(0,2,1)).permute(0,2,1)
    print(f"卷积后维度: {x.shape}")  # 应显示[2,24,16]
    
    x, _ = expert.lstm(x)
    print(f"LSTM后维度: {x.shape}")  # 应显示[2,24,64]
    
    x = expert.proj(x)
    print(f"Proj后维度: {x.shape}")  # 应显示[2,24,32]
    
    output = expert(test_input)
    print(f"最终输出维度: {output.shape}")  # 应显示[2,1]
    # 前向传播检查
    try:
        output = expert(test_input)
        print(f"测试通过，输出维度: {output.shape}")
    except Exception as e:
        print(f"测试失败: {str(e)}")

test_conv_layer()

=== 维度跟踪 ===
卷积后维度: torch.Size([2, 24, 16])
LSTM后维度: torch.Size([2, 24, 64])
Proj后维度: torch.Size([2, 24, 32])
最终输出维度: torch.Size([2, 1])
测试通过，输出维度: torch.Size([2, 1])


In [224]:
# 测试低频专家
def test_low_freq_expert():
    test_input = torch.randn(2, 24, 4)
    expert = ScaleAwareExpert(input_dim=4, scale_type='4hour')
    
    print("=== 低频专家维度跟踪 ===")
    x = expert.proj_in(test_input)
    print(f"投影后维度: {x.shape}")  # [2,24,4]
    
    x = x.permute(1,0,2)
    print(f"转置后维度: {x.shape}")  # [24,2,4]
    
    x = expert.transformer(x)
    print(f"Transformer后维度: {x.shape}")  # [24,2,4]
    
    output = expert(test_input)
    print(f"最终输出维度: {output.shape}")  # [2,1]

test_low_freq_expert()

=== 低频专家维度跟踪 ===
投影后维度: torch.Size([2, 24, 4])
转置后维度: torch.Size([24, 2, 4])
Transformer后维度: torch.Size([24, 2, 4])
最终输出维度: torch.Size([2, 1])


In [225]:
def visualize_training(loss_history, expert_weights, targets, predictions):
    """训练过程可视化分析"""
    plt.figure(figsize=(15,5))
    
    # 损失曲线
    plt.subplot(1,3,1)
    plt.plot(loss_history, label='Training Loss')
    plt.title("Loss Curve")
    plt.xlabel("Epoch"), plt.ylabel("Loss")
    
    # 专家权重分布
    plt.subplot(1,3,2)
    sns.heatmap(np.array(expert_weights), cmap='viridis', 
                xticklabels=['30min','1h','4h','1d'])
    plt.title("Expert Weights Distribution")
    plt.xlabel("Experts"), plt.ylabel("Epoch")
    
    # 预测结果示例
    plt.subplot(1,3,3)
    plt.scatter(targets[:100], predictions[:100], alpha=0.5)
    plt.plot([min(targets), max(targets)], [min(targets), max(targets)], 'r--')
    plt.title("Predictions vs Ground Truth")
    plt.xlabel("True Values"), plt.ylabel("Predictions")
    
    plt.tight_layout()
    plt.show()

In [226]:
# ==================== 训练逻辑 ====================
def hybrid_loss(pred, target, expert_outs, alpha=0.7):
    pred = pred.squeeze(-1)  # (32,1) → (32)
    target = target.squeeze()  # [32] remains
    mse = nn.MSELoss()
    main_loss = mse(pred, target)
    
    # 趋势一致性约束
    expert_tensor = torch.stack(list(expert_outs.values()))  # [num_experts, batch]
    # trends = torch.sign(expert_tensor.detach())  # [num_experts, batch]
    trends = torch.stack([
        torch.sign(expert_outs['30min'].detach()),
        torch.sign(expert_outs['1hour'].detach()), 
        torch.sign(expert_outs['4hour'].detach()),
        torch.sign(expert_outs['1day'].detach())
    ])
    # trends = torch.stack([torch.sign(out.detach()) for out in expert_outs.values()])
    consistency = torch.mean(torch.prod(trends, dim=0))
    
    return alpha*main_loss + (1-alpha)*(1 - consistency)

def print_model_summary(model, device):
    """兼容多输入结构的模型摘要"""
    input_data = [
        torch.randn(1, 24, 4).to(device),  # 30min
        torch.randn(1, 24, 4).to(device),  # 1hour
        torch.randn(1, 24, 4).to(device),  # 4hour
        torch.randn(1, 24, 4).to(device),  # 1day
    ]
    
    # 打印摘要
    summary(model,
            input_data=input_data,
            device=device,
            col_names=["input_size", "output_size", "num_params"],
            depth=3,
            verbose=0
           )
    
    # model.forward = original_forward
    # # 调整模型forward方法临时适配
    # original_forward = model.forward
    # model.forward = lambda x1,x2,x3,x4: original_forward({
    #     '30min': x1, 
    #     '1hour': x2,
    #     '4hour': x3, 
    #     '1day': x4
    # })[0]  # 只返回预测结果
    
    # summary(model, 
    #        input_size=[(24,4)]*4, 
    #        device=str(device),  # 关键修改点
    #        batch_size=1
    #        )
    # model.forward = original_forward


In [ ]:

def train():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = MultiScaleStockDataset(data_path='../data/', scale_factors=['30min', '1hour', '4hour', '1day'])
    
    print(f"数据特征维度: {dataset.data['30min'].shape[1]}")
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    model = MoE(input_dim=4).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    # print_model_summary(model, device)
    # 添加可视化数据收集
    loss_history = []
    expert_weights = []
    predictions = []
    targets = []
    for epoch in range(50):
        epoch_loss = 0
        model.train()
        for batch_idx, (inputs, target) in enumerate(loader):
            inputs = {k:v.to(device) for k,v in inputs.items()}
            target = target.to(device)
            
            pred, expert_outs = model(inputs)
            loss = hybrid_loss(pred, target, expert_outs)
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # 收集预测结果
            with torch.no_grad():
                preds = pred.squeeze().cpu().numpy()
                predictions.extend(preds)
                targets.extend(target.cpu().numpy())
            
            epoch_loss += loss.item()
        # 记录专家权重
        with torch.no_grad():
            dummy_input = {k: torch.randn(1,24,4).to(device) for k in model.experts}
            _, weights = model(dummy_input)
            # expert_weights.append(weights.cpu().numpy()[0])
            expert_weights.append([
                list(model.experts['30min'].parameters())[0].detach().cpu().numpy().mean(),
                list(model.experts['1hour'].parameters())[0].detach().cpu().numpy().mean(),
                list(model.experts['4hour'].parameters())[0].detach().cpu().numpy().mean(),
                list(model.experts['1day'].parameters())[0].detach().cpu().numpy().mean()
            ])
        
        avg_loss = epoch_loss / len(loader)
        loss_history.append(avg_loss)    
        scheduler.step()
        print(f'Epoch {epoch} Loss: {loss.item():.4f}')
    visualize_training(loss_history, expert_weights, targets=targets, predictions=predictions)
    return model
model = train()


数据特征维度: 4


In [ ]:
def analyze_experts(model, device, test_loader):
    """各专家输出分析"""
    expert_outputs = {name: [] for name in model.experts}
    
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = {k:v.to(device) for k,v in inputs.items()}
            _, outputs = model(inputs)
            
            for name, out in outputs.items():
                expert_outputs[name].extend(out.squeeze().cpu().numpy())
    
    plt.figure(figsize=(10,6))
    for name, values in expert_outputs.items():
        sns.kdeplot(values, label=name, alpha=0.6)
    plt.title("Expert Output Distributions")
    plt.xlabel("Prediction Value"), plt.ylabel("Density")
    plt.legend()
    plt.show()

In [ ]:
print("\\n=== Expert Analysis ===")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_dataset = MultiScaleStockDataset(
    data_path='../data/', 
    scale_factors=['30min', '1hour', '4hour', '1day']
)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
analyze_experts(model,device, test_loader)